In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import json
import random
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# List of user-agent headers
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
]

# Define the scopes required for accessing Google Drive.
SCOPES = ['https://www.googleapis.com/auth/drive']

# Function to authenticate and get the Google Drive service object.
def get_drive_service():
    creds = None
    # Check if token.json exists to determine if the user has previously authenticated.
    if os.path.exists('token.json'):
        # If token.json exists, load credentials from it.
        creds = Credentials.from_authorized_user_file('token.json')

    # If credentials are missing or invalid, start the authentication flow.
    if not creds or not creds.valid:
        # Check if credentials are expired and refresh if possible.
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # If no valid credentials exist, initiate the OAuth flow.
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the refreshed or newly obtained credentials to token.json for future use.
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the Google Drive service object using the obtained credentials.
    service = build('drive', 'v3', credentials=creds)
    return service


def extract_content(url, retries=3, delay=1):
    # Define the API endpoint URL
    api_url = "https://h4p8ekalab.execute-api.ap-south-1.amazonaws.com/deploy"

    # Define the payload data to be sent in the POST request
    payload = {
        "url": url
    }

    # Convert the payload to JSON format
    json_payload = json.dumps(payload)

    # Define the headers for the POST request
    headers = {
        'Content-Type': 'application/json'
    }

    for _ in range(retries):
        try:
            # Send the POST request to the API endpoint with a timeout of 10 seconds
            response = requests.post(api_url, headers=headers, data=json_payload, timeout=10)
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Parse the JSON response and extract the body content
                json_data = response.json()
                body_content = json_data.get('body', '')  # Assuming the body is stored under the key 'body'
                return body_content
            else:
                # Print an error message if the request was not successful
                print(f"Error: Unable to fetch URL. Status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred: {e}")
            # Wait for a few seconds before retrying
            time.sleep(delay)

    # Return None if all retries fail
    return None

def extract_content_with_retry(url, retries=15, delay=15):
    for _ in range(retries):
        try:
            content = extract_content(url)
            return content
        except Exception as e:
            print("An error occurred:", e)
        print("Retry:", url)
        time.sleep(delay)
    return None

def extract_act_details(document_text):

    # Split the document text into sentences
    sentences = document_text.split("\n")

    # Define keywords indicating acts or bills
    act_keywords = ["Act", "Bill", "Section"]  # Add more keywords if necessary

    # Extract sentences containing act details
    act_sentences = []
    for sentence in sentences:
        for keyword in act_keywords:
            if keyword in sentence:
                act_sentences.append(sentence.strip())
                break

    act_sentences = " ".join([str(sentence) for sentence in act_sentences])

    return act_sentences

def generate_summary(document_text):

    # Remove act sentences from the document text

    # Create a parser and tokenizer
    parser = PlaintextParser.from_string(document_text, Tokenizer("english"))

    # Summarize the document without act details
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count=15)

    # Join the summary sentences
    summary_text = " ".join([str(sentence) for sentence in summary])

    return summary_text

def generate_summary_with_retry(document_text, retries=5, delay=15):
    for _ in range(retries):
        try:
            content = generate_summary(document_text)
            return content
        except Exception as e:
            print("An error occurred:", e)
        print("Retry:", document_text)
        time.sleep(delay)
    return None

# Function to read document IDs from a text file
def read_doc_ids_from_txt(file_path):
    with open(file_path, 'r') as file:
        doc_ids = file.read().splitlines()
    return doc_ids

# Function to filter DataFrame based on common document IDs
def filter_dataframe(df, common_doc_ids):
    return df[df['doc_id'].isin(common_doc_ids)]

if __name__ == "__main__":
    # Path to the text file and CSV file
    txt_file_path = 'missing_files_allahabad.txt'
    csv_file_path = 'Allahabad.csv'

    # Read document IDs from text file
    with open(txt_file_path, 'r') as file:
        missing_doc_ids = [line.strip() for line in file]

    # Obtain the Google Drive service object
    drive_service = get_drive_service()
    files_uploaded_count = 0
    for index, doc_id in enumerate(missing_doc_ids, start=1):
        try:
            if index % 50 == 0:
                time.sleep(5)
            summary = ""
            url = "https://indiankanoon.org/doc/{}/".format(doc_id)
            content = extract_content_with_retry(url)
            if content:
                content = content.replace("=======================================================", "")

                # Create a file metadata with the desired name and MIME type
                file_metadata = {
                    'name': '{}.txt'.format(doc_id),
                    'parents': ['1arWU0xWTOUv43y8AKLTFHObYgReJKwjT']  # Replace with the ID of the destination folder
                }

                # Create a temporary file to upload to Google Drive
                with open("temp_alh.txt", "w", encoding="utf-8") as temp_file:
                    temp_file.write(content)

                # Upload the file content to Google Drive
                media = MediaFileUpload("temp_alh.txt", mimetype='text/plain')
                uploaded_file = drive_service.files().create(body=file_metadata, media_body=media,
                                                               fields='id').execute()
                files_uploaded_count += 1
                # Summary
                Summary_generated = generate_summary_with_retry(content)
                act_sentences = extract_act_details(content)
                if Summary_generated:
                    summary = Summary_generated
                # Append the summary to the 'Summary' column for the respective row
                if act_sentences:
                    summary = summary + act_sentences

                # Here, you can store the summary in your desired format or location
                print("Index:", index, "Doc ID:", doc_id)
            else:
                print("Failed to fetch content for doc ID:", doc_id)
        except Exception as e:
            print("An error occurred:", e)
            continue

    print("Files uploaded:", files_uploaded_count)


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Index: 1 Doc ID: 161525280
Index: 2 Doc ID: 3190305
Index: 3 Doc ID: 143699489
Index: 4 Doc ID: 27307552
Index: 5 Doc ID: 73444896
Index: 6 Doc ID: 158379558
Index: 7 Doc ID: 8433191
Index: 8 Doc ID: 111193640
Index: 9 Doc ID: 177253929
Index: 10 Doc ID: 194031146
Index: 11 Doc ID: 34647596
Index: 12 Doc ID: 44592
Index: 13 Doc ID: 44593
Index: 14 Doc ID: 29404722
Index: 15 Doc ID: 65056307
Index: 16 Doc ID: 1093178
Index: 17 Doc ID: 67153468
Index: 18 Doc ID: 8433217
Index: 19 Doc ID: 139505222
Index: 20 Doc ID: 199274060
Index: 21 Doc ID: 182496845
Index: 22 Doc ID: 106999372
Index: 23 Doc ID: 154185299
Index: 24 Doc ID: 58764885
Index: 25 Doc ID: 129019478
Index: 26 Doc ID: 1093210
Index: 27 Doc ID: 122728027
Index: 28 Doc ID: 163622492
Index: 29 Doc ID: 90222176
Index: 30 Doc ID: 44646
Index: 31 Doc ID: 1093225
Index: 32 Doc ID: 190885483
Index: 33 Doc ID: 153136751
Index: 34 Doc ID: 138456687
Index: 35 Doc ID: 109096561
Index: 36 Doc ID: 185642607
Index: 37 Doc ID: 185642613
Index

/usr/local/lib/python3.10/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (31) is lower than number of sentences (896). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Index: 143 Doc ID: 129019733
Index: 144 Doc ID: 60862292
Index: 145 Doc ID: 98611033
Index: 146 Doc ID: 183545691
Index: 147 Doc ID: 144748380
Index: 148 Doc ID: 2142044
Index: 149 Doc ID: 84979550
Index: 150 Doc ID: 84979551
Index: 151 Doc ID: 84979552
Index: 152 Doc ID: 187740002
Index: 153 Doc ID: 192982883
Index: 154 Doc ID: 65056610
Index: 155 Doc ID: 14724968
Index: 156 Doc ID: 52473708
Index: 157 Doc ID: 92319601
Index: 158 Doc ID: 199274353
Index: 159 Doc ID: 165719926
Index: 160 Doc ID: 7384951
Index: 161 Doc ID: 56668034
Index: 162 Doc ID: 57716612


/usr/local/lib/python3.10/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (23) is lower than number of sentences (989). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


...

Index: 725 Doc ID: 97563731
Index: 726 Doc ID: 123778132
Failed to fetch content for doc ID: 195081303
Index: 728 Doc ID: 136361047
Index: 729 Doc ID: 18920535
Index: 730 Doc ID: 68203611
Index: 731 Doc ID: 147895387
Index: 732 Doc ID: 130069596
Index: 733 Doc ID: 183546985
Index: 734 Doc ID: 147895406
Index: 735 Doc ID: 93369455
Index: 736 Doc ID: 181449842
Index: 737 Doc ID: 156284023
Index: 738 Doc ID: 15774841
Index: 739 Doc ID: 55620732
Index: 740 Doc ID: 114340989
Index: 741 Doc ID: 143701116
Index: 742 Doc ID: 52475008
Index: 743 Doc ID: 53523585
Index: 744 Doc ID: 126923911
Index: 745 Doc ID: 24163463
Index: 746 Doc ID: 178304139
Index: 747 Doc ID: 39892108
Index: 748 Doc ID: 119583884
Index: 749 Doc ID: 97563790
Index: 750 Doc ID: 115389591
Index: 751 Doc ID: 118535322
Index: 752 Doc ID: 45135011
Index: 753 Doc ID: 1094827
Index: 754 Doc ID: 103855278
Index: 755 Doc ID: 57717936
Index: 756 Doc ID: 168866995
Index: 757 Doc ID: 89175223
Index: 758 Doc ID: 136361146
Index: 759 Do

/usr/local/lib/python3.10/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (29) is lower than number of sentences (998). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Index: 831 Doc ID: 13677962
Index: 832 Doc ID: 2143635
Index: 833 Doc ID: 141604244
Index: 834 Doc ID: 40940949
Index: 835 Doc ID: 105952662
Index: 836 Doc ID: 181450132
Index: 837 Doc ID: 46495
Index: 838 Doc ID: 24163747
Index: 839 Doc ID: 175158691
Index: 840 Doc ID: 135312805
Index: 841 Doc ID: 198227366
Index: 842 Doc ID: 44086700
Index: 843 Doc ID: 178304429
Index: 844 Doc ID: 181450162
Index: 845 Doc ID: 164672947
Index: 846 Doc ID: 44086709
Index: 847 Doc ID: 126924229
Index: 848 Doc ID: 83932613
Index: 849 Doc ID: 190887368
Index: 850 Doc ID: 45135304
Index: 851 Doc ID: 114341322
Index: 852 Doc ID: 1095115
Index: 853 Doc ID: 44086732
Index: 854 Doc ID: 136361426
Index: 855 Doc ID: 80786899
Index: 856 Doc ID: 140555736
Index: 857 Doc ID: 146847194
Index: 858 Doc ID: 144750047
Index: 859 Doc ID: 46562
Index: 860 Doc ID: 111195619
Index: 861 Doc ID: 78689762
Index: 862 Doc ID: 92321252
Index: 863 Doc ID: 127972840
Index: 864 Doc ID: 8435176
Index: 865 Doc ID: 178304493
Index: 866

/usr/local/lib/python3.10/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (28) is lower than number of sentences (924). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Index: 1130 Doc ID: 94419022
Index: 1131 Doc ID: 120633423
Index: 1132 Doc ID: 51427410
Index: 1133 Doc ID: 141604947
Index: 1134 Doc ID: 110147668
Index: 1135 Doc ID: 1095764
Index: 1136 Doc ID: 155236434
Index: 1137 Doc ID: 154187864
Index: 1138 Doc ID: 18921561
Index: 1139 Doc ID: 129022041
Index: 1140 Doc ID: 69253208
Index: 1141 Doc ID: 29407325
Index: 1142 Doc ID: 118536292
Index: 1143 Doc ID: 47206
Index: 1144 Doc ID: 199276648
Index: 1145 Doc ID: 195082345
Index: 1146 Doc ID: 91273327
Index: 1147 Doc ID: 49330288
Index: 1148 Doc ID: 108050547
Index: 1149 Doc ID: 36747379
Index: 1150 Doc ID: 195082359
Index: 1151 Doc ID: 164673656
Index: 1152 Doc ID: 45135996
Index: 1153 Doc ID: 192985217
Index: 1154 Doc ID: 14727304
Index: 1155 Doc ID: 54573194
Index: 1156 Doc ID: 1095822
Index: 1157 Doc ID: 24164501
Index: 1158 Doc ID: 79739038
Index: 1159 Doc ID: 147896482
Index: 1160 Doc ID: 158382243
Index: 1161 Doc ID: 115390627
Index: 1162 Doc ID: 169916581
Index: 1163 Doc ID: 112244902
I

In [ ]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 25.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=ea4e79f741e565f1ba1cbe6f1fe9d8622395ab59c32811930f7e75136fde3321
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=01c04f2787cef569596e0d059c0043a4a6bb89853540740c418242d8e6c5da7c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 27.8 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=9f9e46efc6b9f97ee4f3f5e72936936ac66b72272439c91c71c68e3b33887d9f
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0604650fb07726bc6db71d0096b7fc1caa6d515123630be52064efd62280b2bd
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
